# Wykład 14 - Wielowątkowość - `Coroutines`

In [4]:
import java.util.concurrent.Callable
import java.util.concurrent.ExecutionException
import java.util.concurrent.ExecutorService
import java.util.concurrent.Executors
import java.util.concurrent.Future

val executor = Executors.newFixedThreadPool(4)
val future: Future<Double> = executor.submit(Callable<Double> {
    Math.sqrt(15.64)
})

try {
    while (!future.isDone) println("waiting")
    println(future.get())
} catch (ee: ExecutionException) {
    System.err.println("task threw an exception")
    ee.printStackTrace()
} catch (ie: InterruptedException) {
    System.err.println("interrupted while waiting")
}
executor.shutdown()

waiting
waiting
3.954743986657038


In [9]:
import java.util.concurrent.CompletableFuture
import java.util.function.Supplier // interfejs funkcyjny

val executor = Executors.newFixedThreadPool(4)
val future = CompletableFuture
    .supplyAsync(Supplier { Math.sqrt(15.64)}, executor)
    .thenApply { // callback
        println("$it")}

3.954743986657038


Funkcja `thenApply` zostanie wywołana po zakończeniu obliczeń. Występuje jednak kilka ograniczeń
- jeżeli konieczna jest implementacja komunikacji między wykonywanymi zadaniami konieczna jest synchronizacja
- jeżeli jeden `callback` potrzebuje innego wymagane jest zagnieżdżenie co prowadzi do sytuacji znanej jako **callback hell**.

Kotlin wprowadza rozwiązanie powyższych problemów, jednocześnie zachowując imperatywny styl programowania - **Coroutines**. Jest to koncepcja (**continuasions**) - abstrakcja opisująca aktualny stan programu (wątku) w sposób umożliwiający znowienie działania po zawieszeniu.

## 14.1 Funkcje zawieszenia - `suspend fun`

Zawieszenie jest zdolnością funkcji do zawieszenia wykonania i wznowienia w późniejszym czasie.

Gdy wykonanie jest zawieszone, kompilator zapisuje stan wraz ze wszystkimi niezbędnymi informacjami (przykładowo lokalne zmienne) - powstały obiekt jest niewielki. Jeżeli funkcja jest zawieszona, inna funkcja może zostać wykonana na danym wątku.

Nie występuje sytuacja zablokowania wątku

```kotlin
suspend fun delay(timeMillis: Long) { ... }
```

## 14.2 Budowniczy - `Coroutine.Builder`

Aby wykonać funkcje zawieszoną musi ona zostać umieszczona w odpowiednim kontenerze - tym kontenerem jest `Coroutine`. Każda `Coroutine` musi zostać wykonana na wątku - ale nie musi to być jeden określony wątek. Przykłady:
- `launch` - gdy nie interesuje nas wartość zwracana
- `async`
- `runBlocking` - gdy chcemy zablokować wykonanie wątku

In [ ]:
import kotlinx.coroutines.*

fun main(){
    GlobalScope.launch { 
        delay(1000L)
        println("World!")
    }
    println("Hello")
    Thread.sleep(2000)
}

`launch` tworzy `Coroutine` i rozpoczyna wykonanie wyrażenia lambda. Wpierw wykonujemy `delay` aby zawiesić wykonanie. W tym samym czasie kod poza `Coroutine` zostaje wykonany bez blokowania. Gdy nie użyjemy `Thread.sleep` funkcja `main`, a tym samym JVM, zakończy działanie.

Kolejną opcją jest `runBlocking`, który tworzy `Coroutine` i blokuje aktualny wątek dopóki utworzona `Coroutine` nie zakończy działania

In [ ]:
fun main() {
    runBlocking {
        delay(1000)
        println("Hello World")
    }
    println("Done")
}

In [ ]:
fun main() {
    suspend fun a() {
        delay(100)
        println("a")
    }
    suspend fun b() {
        delay(100)
        println("b")
    }
    runBlocking {
        a()
        b()
        println("Done")
    }
}


In [ ]:
fun main() {
    suspend fun a(): Int = {
        delay(100)
        Random.nextInt()
    }
    suspend fun b(a: Int) {
        println("Using $a")
    }
    runBlocking {
        val a = a()
        b(a)
    }
}


In [ ]:
fun main() {
    runBlocking {
        println("Parent starting")
        launch {
            println("Child a starting")
            delay(100)
            println("Child a complete")
        }
        launch {
            println("Child b starting")
            delay(100)
            println("Child b complete")
        }
        println("Parent complete")
    }
    println("Parent returned")
}

## 14.3 `Job`

Gdy `Coroutine` jest utworzony przy wykorzystaniu `launch`, zwracana jest wartość typu `Job`. Jedną z głównych funkcji `Job` jest możliwość zakończenia wykonania.

In [ ]:
fun main() {
    var x = 0
    val job = GlobalScope.launch {
        while (true) {
            x += 1
            delay(10)
            println(x)
        }
    }
    runBlocking {
        delay(100)
        job.cancel()
    }
}

Następną jest możliwość oczekiwania na zakończenie działania za pomocą metody `join`.

In [ ]:
fun main() {
    val job = GlobalScope.launch {
        delay(1000L)
        println("Hello")
    }
    runBlocking {
        job.join()
        println("World")
    }
}

In [ ]:
fun main() = runBlocking {
    repeat(100) { // launch a lot of coroutines
        launch {
            delay(5000L)
            print(".")
        }
    }
}

## 14.4 `Context`

Każda `Coroutine` posiada powiązany `Context` - jest to niemutowalna kolekcja kluczy i wartości - dzięki temu możemy nadać nazwę każdej `Coroutine` lub określić który wątek z powiązanego `ThreadPool` powinien wykonać zadanie.

## 14.5 `Async`

In [ ]:
fun main() {
    runBlocking {
        val deferred = async {
            println("Computing value...")
            delay(1000)
            10
        }
        val result = deferred.await()
        println("The result is $result")
    }
}

In [ ]:
import kotlinx.coroutines.*
import kotlin.system.measureTimeMillis

fun main() {
    suspend fun fetchOrder(): String {
        delay(3000)
        return "Order 123"
    }
    suspend fun fetchAddress(): String {
        delay(2000)
        return "10 Downing Street"
    }
    suspend fun fetchUsername(): String {
        delay(1000)
        return "Prime Minister"
    }
    println("Starting")
    val time = measureTimeMillis {
        runBlocking {
            val order = async {
                fetchOrder()
            }
            val address = async {
                fetchAddress()
            }
            val username = async {
                fetchUsername()
            }
            println("Shipping ${order.await()} to ${username.await()}at ${address.await()}")
        }
    }
    println("Completed in $time ms")
}


## 14.6 `Dispatcher`

Jeżeli chcemy mieć kontrolę nad tym który wątek z puli dana `Coroutine` jest wykonana możemy wykorzystać `Dispatcher`
- `Dispatcher.Default` - domyślny wykorzystuje `ThreadPool`, którego wielkość jest równa liczbie rdzeni CPU
- `Dispatcher.IO` - rekomendowany do operacji IO, wykorzystuje `ThreadPool` który oże zostać rozszerzony
- `Dispatcher.Main` - wykorzystuje wątek główny UI.

In [ ]:
fun main() {
    runBlocking(Dispatchers.IO) {
        println(Thread.currentThread().name)
    }
}